In [18]:
import tensorflow as tf
import keras

In [19]:
TRAINING_DIR = 'F:/Xray_Gaus/train'
TEST_DIR='F:/Xray_Gaus/test'
VAL_DIR='F:/Xray_Gaus/val'

In [20]:
num_classes = 2
IMG_SIZE = 224
BATCH_SIZE = 64
input_shape = (224,224, 3)

learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 10


In [21]:
from keras_preprocessing.image import ImageDataGenerator

# Training IDG
train_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2)

# Training Gen
train_gen = train_idg.flow_from_directory(
    TRAINING_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    classes=['normal','COVID-19'],
    class_mode='binary',
    subset='training'
)

Found 5805 images belonging to 2 classes.


In [22]:
val_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True)

# Training Gen
val_gen = val_idg.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary',
    classes=['normal','COVID-19'],
    subset='training'
)

Found 905 images belonging to 2 classes.


In [23]:
test_idg = ImageDataGenerator(rescale=1./255)

# Test Gen
test_gen = test_idg.flow_from_directory(
        TEST_DIR,
        target_size=(IMG_SIZE, IMG_SIZE),
        shuffle=True,
        class_mode='binary',
    classes=['normal','COVID-19'],
        #subset='validation'
)

Found 905 images belonging to 2 classes.


In [24]:
pretrained_model = keras.models.load_model("F:/Xray_Gaus/CNN_binary_wt")


In [25]:
STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = val_gen.n // val_gen.batch_size

history=pretrained_model.fit(x = train_gen,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = val_gen,
          validation_steps = STEP_SIZE_VALID,
          epochs = 3)

Epoch 1/3
90/90 [==============================] - 258s 3s/step - loss: 0.0575 - accuracy: 0.9831 - val_loss: 0.0974 - val_accuracy: 0.9788
Epoch 2/3
90/90 [==============================] - 284s 3s/step - loss: 0.0383 - accuracy: 0.9883 - val_loss: 0.1547 - val_accuracy: 0.9810
Epoch 3/3
90/90 [==============================] - 295s 3s/step - loss: 0.0302 - accuracy: 0.9904 - val_loss: 1.2592 - val_accuracy: 0.5692


In [26]:
results = pretrained_model.evaluate(test_gen, batch_size=128)

29/29 [==============================] - 20s 689ms/step - loss: 0.1767 - accuracy: 0.9271


In [27]:
y_pred = pretrained_model.predict(test_gen, verbose=1)
#y_pred = np.argmax(y_pred, axis = 1)

#y_pred=model.predict(X_test)

for i in range(len(y_pred)):
    if y_pred[i]>=0.5:
        y_pred[i]=1
    else:
        y_pred[i]=0

29/29 [==============================] - 12s 409ms/step


In [28]:
y_true=test_gen.labels

In [29]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.98      0.92      0.95       885
           1       0.01      0.05      0.02        20

    accuracy                           0.90       905
   macro avg       0.50      0.48      0.48       905
weighted avg       0.96      0.90      0.93       905

